<a href="https://colab.research.google.com/github/YashPatel1604/AI-workshop-UTD/blob/main/HousingDataWithNueralNetworks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential 
from keras.layers import Activation, Dense, Input 


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
split_index = train.iloc[-1].Id
na = train.isna().sum()
print(na[na>0])
print('----------')
df = pd.concat((train,test),sort = False)
df.set_index('Id',drop=True,inplace=True)

LotFrontage      259
Alley           1369
MasVnrType         8
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64
----------


In [3]:
def removing(x):
  
  x.Alley.fillna('NA',inplace = True)
  x.BsmtQual.fillna('NA',inplace = True)
  x.BsmtCond.fillna('NA',inplace = True)
  x.BsmtExposure.fillna('NA',inplace = True)
  x.BsmtFinType1.fillna('NA',inplace = True)
  x.BsmtFinType2.fillna('NA',inplace = True)
  x.FireplaceQu.fillna('NA',inplace = True)
  x.GarageType.fillna('NA',inplace = True)
  x.GarageFinish.fillna('NA',inplace = True)
  x.GarageQual.fillna('NA',inplace = True)
  x.GarageCond.fillna('NA',inplace = True)
  x.PoolQC.fillna('NA',inplace = True)
  x.Fence.fillna('NA',inplace = True)
  x.MiscFeature.fillna('NA',inplace = True)

  x.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
  # x.drop(['Alley','PoolQC','Fence','MiscFeature'],axis = 1,inplace = True)
  columns = x.columns.values
  GarageNull = pd.DataFrame(x['GarageYrBlt'].isnull().values)
  for y in GarageNull:
    yin = GarageNull.index
    x['GarageYrBlt'].iloc[yin] = (x['YearBuilt'].iloc[yin])
  for column in columns:
    if x[column].dtype != np.int64 and x[column].dtype != np.float64:
      if x[column].isna().sum()>200:
        x[column].fillna(value ='No Feature',inplace = True)
      else:
        x[column].fillna(value=x[column].mode(),inplace=True)
    else:
      x[column].fillna(value = x[column].mean(),inplace = True)

def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = str(x)
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))
def main(df):
  removing(df)
  handle_non_numerical_data(df)
  # one_hot_encoding(df)

def add_missing_dummy_columns(test,train):
  missing_cols = set(train.columns) - set(test.columns)
  missing_cols2 = set(test.columns) - set(train.columns)
  for c in missing_cols:
    test[c] = 0

  for c in missing_cols2:
    train[c] = 0

In [4]:
main(df)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
encoded_df = pd.get_dummies(df)

In [6]:
from sklearn.preprocessing import minmax_scale
for column in encoded_df.columns:
  if column != 'SalePrice':
    encoded_df[column] = pd.Series(minmax_scale(encoded_df[column]),index=encoded_df.index)

In [13]:
from sklearn.preprocessing import MinMaxScaler
target_col = encoded_df.loc[:split_index].SalePrice
target_scaler = MinMaxScaler()
Y = np.array(target_col.to_numpy())
Y = Y.reshape(Y.size,1)
target_scaler.fit(Y)
Y = target_scaler.transform(Y)
Y = Y.reshape(Y.size)
encoded_df.drop('SalePrice',axis=1,inplace=True)

In [14]:
layer1 = Dense(units = 14,activation = 'sigmoid')
layer2 = Dense(units = 3,activation = 'sigmoid')
layer3= Dense(units = 4,activation = 'sigmoid')
layer4=Dense(units = 1,activation='sigmoid')

In [15]:
model = Sequential([layer1,layer2,layer3,layer4])

In [16]:
train_df = encoded_df.loc[:split_index]
test_df = encoded_df.loc[split_index+1:]

In [17]:
model.compile(optimizer='sgd', loss='mse')
model.fit(train_df,Y,epochs = 100)

Epoch 1/100
46/46 [==============================] - 1s 3ms/step - loss: 0.0352
Epoch 2/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0309
Epoch 3/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0275
Epoch 4/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0248
Epoch 5/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0227
Epoch 6/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0210
Epoch 7/100
46/46 [==============================] - 0s 3ms/step - loss: 0.0195
Epoch 8/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0184
Epoch 9/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0174
Epoch 10/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0166
Epoch 11/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0160
Epoch 12/100
46/46 [==============================] - 0s 2ms/step - loss: 0.0154
Epoch 13/100
46/46 [=================

In [18]:
y_pred = model.predict(test_df)

In [19]:
final=target_scaler.inverse_transform(y_pred.reshape(-1,1))

In [20]:
final

array([[180584.45],
       [180449.45],
       [181472.62],
       ...,
       [180154.05],
       [180824.1 ],
       [181348.17]], dtype=float32)

In [21]:
final.size

1459

In [22]:
final = final.flatten()

In [23]:
flat_list = [x for x in final]
prediction = pd.DataFrame({'id': range(1461,2920),'SalePrice':flat_list})
prediction.head()

,id,SalePrice
0,1461,180584.453125
1,1462,180449.453125
2,1463,181472.625000
3,1464,182033.250000
4,1465,181520.375000


In [24]:
pd.DataFrame(prediction).to_csv("prediction_file.csv",index = False)